# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:] #since the first 81 characters are copyright-related: 
    # [YEAR DATE 1989] © Twentieth Century Fox Film Corporation. All rights reserved.
print(text[:1000])


Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


Moe_Szyslak: Ah, isn't that nice. Now, there is a politician who cares.
Barney_Gumble: If I ever vote, it'll be for him. (BELCH)


Barney_Gumble: Hey Homer, how's your neighbor's store doing?
Homer_Simpson: Lousy. He just sits there all day. He'd have a great job if he didn't own the place. (CHUCKLES)

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]

    # scenes is defined by a paragraph text split with double \n\n - scenes = text.split('\n\n')
    # for scene in scenes --- scene.count number of single \n's - that's = number of sentences in each scene as an array matrix


Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262


In [3]:
print ('First Scene:\n', scenes[0],'\n\n','Second Scene:\n',scenes[1])


First Scene:
 
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills. 

 Second Scene:
 
Moe_Szyslak: Ah, isn't that nice. Now, there is a politician who cares.
Barney_Gumble: If I ever vote, it'll be for him. (BELCH)


In [4]:
print('To count the number of sentences  per scene, we count the number of sentence breaks \n in for each scene in scenes')
# sentence_count_scene = [scene.count('\n') for scene in scenes]
print('\nThe number of sentences in each of the 262 scenes is: \n', sentence_count_scene)

To count the number of sentences  per scene, we count the number of sentence breaks 
 in for each scene in scenes

The number of sentences in each of the 262 scenes is: 
 [8, 2, 5, 16, 4, 21, 76, 54, 56, 4, 7, 12, 6, 14, 4, 20, 11, 1, 3, 17, 5, 3, 10, 24, 51, 4, 4, 9, 2, 11, 4, 2, 36, 30, 3, 8, 12, 8, 14, 33, 7, 6, 12, 3, 4, 12, 3, 65, 4, 19, 11, 19, 13, 3, 10, 9, 11, 1, 16, 1, 21, 23, 9, 22, 10, 18, 56, 66, 8, 24, 15, 10, 6, 21, 15, 10, 21, 18, 29, 21, 2, 10, 9, 9, 3, 4, 15, 5, 7, 13, 42, 6, 77, 7, 24, 33, 20, 11, 35, 7, 15, 91, 12, 10, 24, 19, 11, 12, 3, 1, 16, 25, 9, 18, 39, 6, 9, 5, 9, 13, 9, 9, 38, 11, 5, 15, 12, 5, 6, 16, 41, 17, 38, 7, 3, 66, 2, 1, 13, 1, 1, 3, 24, 7, 2, 5, 17, 6, 32, 5, 2, 11, 5, 11, 13, 5, 5, 10, 9, 3, 14, 5, 3, 5, 26, 38, 18, 2, 9, 5, 2, 9, 31, 13, 19, 15, 7, 62, 13, 9, 18, 68, 14, 30, 13, 26, 27, 28, 27, 7, 16, 21, 17, 1, 15, 7, 33, 23, 9, 25, 3, 9, 2, 3, 46, 16, 4, 23, 14, 14, 36, 3, 21, 4, 1, 1, 5, 20, 76, 8, 42, 4, 4, 8, 31, 1, 21, 2, 8, 12, 2, 28, 1, 11,

In [5]:

print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]

# for each sentence in each scene that exists in all scenes - create an array of an individual sentence and store in
   # sentences array
    
print('The first 3 sentences are: \n\n', sentences[:3])

Average number of sentences in each scene: 15.251908396946565
The first 3 sentences are: 

 ['', "Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.", 'Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.']


In [6]:
# First scene looks like:
#First Scene:
 
#Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
#Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.

# The first sentence is just empty \n
# The second and third sentence contain the first 2 lines of the scene.

print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]

#for each sentence in all sentences array, split it based on default empty space - find out the length of how many splits
  # were made because the split shows how many words there are. 
    
    #since we used the len() function and stored into word_count_sentence -- this is a number, which means we can do
    # numpy math with word_count_sentence -- including np.average(word_count_sentence)
    
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

#Recall again first sentence:
# Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
print('The number of words in the first sentence (excluding the initial space) - is \n\n',
      "Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\n\n", "= ", word_count_sentence[1], 'words')

Number of lines: 4258
Average number of words in each line: 11.50164396430249
The number of words in the first sentence (excluding the initial space) - is 

 Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.

 =  11 words


In [7]:

# Recall that up top we specified:
# view_sentence_range = (0, 10)
view_sentence_range = (0, 20)

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

# Example join() command
# s = "-";
#seq = ("a", "b", "c"); # This is sequence of strings.
#print s.join( seq )
# output:
# a-b-c

# so in the above print statement, we are adding a line break \n after we split the text into array elements 
    # with the same \n character


The sentences 0 to 20:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


Moe_Szyslak: Ah, isn't that nice. Now, there is a politician who cares.
Barney_Gumble: If I ever vote, it'll be for him. (BELCH)


Barney_Gumble: Hey Homer, how's your neighbor's store doing?
Homer_Simpson: Lousy. He just sits there all day. He'd have a great job if he didn't o

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [8]:
import numpy as np
import problem_unittests as tests




words = text.split()
print('First 200 Words\n\n', words[:200])

words_no_duplicates = set(words)

print('unique words_no_duplicates (random order) by applying set(text)\n\n', words_no_duplicates)
print ('number of unique words:\n', len(words_no_duplicates))

words_no_duplicates = sorted(words_no_duplicates)
print('\n words sorted version of words_no_duplicates\n\n', words_no_duplicates)

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    # vocab_to_int = set of paired word/integer dictionary for each integer/word in the enumerated text 
        #- beginning with 1 for the first word (instead of 0, in case we want to left pad with 0's later)
    
    #vocab_to_int = {word: ii for ii, word in enumerate(words_no_duplicates, 1)}
    
    
    
    vocab_to_int = dict((word, i) for i, word in enumerate(words_no_duplicates))
    print("\nDictionary: vocab_to_int\n\n", vocab_to_int, '\n')
    
    print('\nSorted vocab_to_int dictionary using\n sorted(vocab_to_int.items(), reverse=false) \n\n', sorted(vocab_to_int.items(), reverse=False))
    
    print("\n\nThe letter Q corresponds with the integer using\n word_to_int.get('Q'): ", vocab_to_int.get("Q", None))
    
    int_to_vocab = dict((i, word) for i, word in enumerate(words_no_duplicates))
    print("\nDictionary: int_to_vocab\n\n", int_to_vocab)


   # int_to_vocab = {ii: word for ii, word in enumerate(words_no_duplicates, 1)}
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

First 200 Words

 ['Moe_Szyslak:', '(INTO', 'PHONE)', "Moe's", 'Tavern.', 'Where', 'the', 'elite', 'meet', 'to', 'drink.', 'Bart_Simpson:', 'Eh,', 'yeah,', 'hello,', 'is', 'Mike', 'there?', 'Last', 'name,', 'Rotch.', 'Moe_Szyslak:', '(INTO', 'PHONE)', 'Hold', 'on,', "I'll", 'check.', '(TO', 'BARFLIES)', 'Mike', 'Rotch.', 'Mike', 'Rotch.', 'Hey,', 'has', 'anybody', 'seen', 'Mike', 'Rotch,', 'lately?', 'Moe_Szyslak:', '(INTO', 'PHONE)', 'Listen', 'you', 'little', 'puke.', 'One', 'of', 'these', 'days', "I'm", 'gonna', 'catch', 'you,', 'and', "I'm", 'gonna', 'carve', 'my', 'name', 'on', 'your', 'back', 'with', 'an', 'ice', 'pick.', 'Moe_Szyslak:', "What's", 'the', 'matter', 'Homer?', "You're", 'not', 'your', 'normal', 'effervescent', 'self.', 'Homer_Simpson:', 'I', 'got', 'my', 'problems,', 'Moe.', 'Give', 'me', 'another', 'one.', 'Moe_Szyslak:', 'Homer,', 'hey,', 'you', 'should', 'not', 'drink', 'to', 'forget', 'your', 'problems.', 'Barney_Gumble:', 'Yeah,', 'you', 'should', 'only', 'drin

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [9]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    symbol_dict = {'.':'||Period||', ',':'||Comma||', '"':'||Quotation-Mark||', ';':'Semicolon', '!':'||Exclamation-Mark||', '?':'Question-Mark',
                  '(':'||Left-Parenthesis||', ')':'||Right-Parenthesis||', '--':'||Dash||', '\n':'||Return||'}
    return symbol_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)


Dictionary: vocab_to_int

 {'Larson!': 2879, 'unsanitary.': 10972, 'big,': 5309, 'boxing.': 5436, 'Fonda,': 2144, 'Sesame': 3929, 'prayers.': 9201, 'Shakespeare': 3944, 'Lenny': 2897, 'koi': 7972, 'grim': 7286, 'Ling': 2923, 'gimme': 7167, 'elaborate!': 6580, 'MA': 2982, 'YOU...': 4684, 'hitchhike.': 7537, 'dance': 6142, 'roller': 9579, 'candidate': 5628, 'Aww.': 1184, 'GUY': 2244, 'This': 4289, 'Joe.': 2715, 'onions': 8812, 'sign': 9929, 'Avenue': 1178, 'say,': 9699, 'CHARACTER)': 1498, "tryin'": 10878, 'NOT': 3281, 'Statue': 4073, 'moolah-stealing': 8485, 'found.': 7003, 'results.': 9524, 'Put': 3621, 'as?': 5032, 'Holy': 2522, 'Every': 2016, 'eyeball': 6732, 'universe': 10963, 'Keeps': 2764, 'usually': 11006, 'card': 5646, 'ping-pong': 9056, 'mom': 8465, 'ahead,': 4854, 'Hospital...': 2548, 'I/YOU': 2601, 'woo-hoo!': 11322, '(CONFUSED)': 308, 'city.': 5811, 'trench': 10840, 'Obese': 3390, 'Easy,': 1957, 'beautiful!': 5219, 'moved': 8522, 'LITTLE': 2842, 'pushing': 9334, 'happy!': 7

KeyError: '||return||'

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


C:\Users\Allen\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearingRate)`

In [13]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    
    
    
        # inputs needs to be 2-dimensional and be of type integer.
        # Why? 
    inputs_ = tf.placeholder(tf.int32, [None, None], name='input')
    targets_ = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
        
    return inputs_, targets_, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [14]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm  = tf.contrib.rnn.BasicLSTMCell(rnn_size)

    
    Cell = tf.contrib.rnn.MultiRNNCell([lstm])
    # rnn_size is passed into this function above
    
    # Getting an initial state of all zeros
    InitialState = Cell.zero_state(batch_size, tf.float32)
    InitialState = tf.identity(InitialState, name="initial_state")
    
    
    return Cell, InitialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [15]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    
    
        # We don't initialize (embedding) with a placeholder. Instead, we initialize with random uniform distribution
        # hence we use tf.Variable instead of tf.placeholder
        
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim)))
    embed = tf.nn.embedding_lookup(embedding, input_data)
            # looks up the embedding table that is of embed_dim dimensions for the specific input_data
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [16]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype = tf.float32)
    final_state = tf.identity(state, name = 'final_state')
    return outputs, final_state

    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [17]:
def build_nn(cell, rnn_size, input_data, vocab_size):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    # Apply embedding to input_data using your get_embed(input_data, vocab_size, embed_dim) function.
    embed = get_embed(input_data, vocab_size, rnn_size)
          # in place of (embed_dim) - we use (rnn_size) - as the 3rd parameter of the get_embed() function
    
    # after applying embedding to input data...
    # we feed the cell and the embed into the build_rnn() function, and store results into (outputs) and (finalstate)
    outputs, finalstate = build_rnn(cell = cell, inputs = embed)
    
    
    #Apply a fully connected layer with a linear activation and vocab_size as the number of outputs.
    logits = tf.contrib.layers.fully_connected(outputs, 
                                                vocab_size, 
                                                activation_fn=None,
                                               #weights_initializer = tf.truncated_normal_initializer(stddev=0.1),
                                               #biases_initializer=tf.zeros_initializer()
                                              )
    #weights = tf.Variable(tf.truncated_normal(shape=[rnn_size, vocab_size], 
     #                                         mean=0.0, 
      #                                        stddev=0.1, 
      #                                        dtype=tf.float32, 
          #                                    seed=123)) 
    
    #biases = tf.Variable(tf.zeros(shape=[vocab_size], dtype=tf.float32)) 
    
   # def mul_fn(current_input):
     #   return tf.matmul(current_input, weights) + biases

   # logits = tf.map_fn(mul_fn, outputs)

    return logits, finalstate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [18]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    n_batches = len(int_text)//(batch_size*seq_length)

    # only full batches
    int_text = int_text[:n_batches*batch_size*seq_length+1]

    # Initialize result & calculate skip distance
    result=np.ndarray(shape=(n_batches,2,batch_size,seq_length), dtype=int)
    skipdistance = n_batches*seq_length    

    # First loop steps at batch_size * seq_length
    for b in range(n_batches):
        batch_idx = b*seq_length
        x , y= [], []
        for bb in range(batch_size):
            idx = batch_idx + (bb*skipdistance) # get starting index for batch
            x_idx = idx
            y_idx = idx+1          
            result[b][0][bb] = int_text[x_idx:x_idx+seq_length]
            result[b][1][bb] = int_text[y_idx:y_idx+seq_length]                   
    return result

   # return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [19]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = .01
# Show stats for every n number of batches
show_every_n_batches = 5

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [71]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/33   train_loss = 8.815
Epoch   0 Batch    5/33   train_loss = 6.745


InvalidArgumentError: Received a label value of 6780 which is outside the valid range of [0, 6780).  Label values: 5128 5707 5998 917 1992 5409 5128 6595 5508 262 161 4691 3916 1992 5360 3443 2928 4227 2113 1867 1455 2889 3181 5360 101 5805 5128 2246 1992 5360 5989 2928 3755 262 3165 1992 1538 1867 2928 612 6539 3181 5360 5989 262 1339 111 2246 3852 1992 5360 2864 3430 2113 3513 1993 242 5570 5128 1547 2928 4615 5101 1672 4552 1329 5998 3451 3181 4552 4180 5069 5998 2342 262 1086 5642 3165 6282 2618 1992 1152 3430 2113 4500 1993 3068 5570 5195 5598 1992 3430 2113 3513 1993 6780 5312 4406 5607 1992 6282 5508 4615 3728 5128 6282 5508 5617 191 4503 3547 1903 3465 5998 5741 4615 1720 1373 4615 2878 1992 3595 657 1198 2555 1992 5360 4852 5847 5998 4539 6523 3181 5360 101 3430 3521 1993 535 5440 3798 317 2875 5128 5409 5128 5300 262 5445 2113 1149 6539 1992 5360 5566 5594 5128 3265 3755 1759 3443 5594 5128 3513 5128 2928 5894 5441 4838 262 2618 2113 6717 6172 3913 4452 2928 5894 2655 5128 50

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [24]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    #Get tensors from loaded_graph using the function get_tensor_by_name(). 
    
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [25]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    vocab_list = list(int_to_vocab.values())
    print('vocab_list\n\n', vocab_list)
    
    print('\nprobabilities\n\n', probabilities)
    
    #predicted_word = np.random.choice(list(int_to_vocab.values()), probabilities)
    best_word= np.random.choice(vocab_list, 1, p=probabilities)
    
    print(best_word)
    #use np.random.choice on the probabilities array - and within that array, look at values >=.2
    print(best_word[0])
    return best_word[0]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

vocab_list

 ['this', 'is', 'a', 'test']

probabilities

 [ 0.1   0.8   0.05  0.05]
['is']
is
Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [26]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

OSError: File ./save.meta does not exist.

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.